In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#%matplotlib inline

import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt

In [ ]:
import json

from ochre.utils import read_texts

datasets = '/home/jvdzwaan/data/icdar2017st/eng_monograph/datadivision-sample.json'
data_dir = '/home/jvdzwaan/data/icdar2017st/eng_monograph/aligned/'

with open(datasets) as d:
    division = json.load(d)
print(len(division['train']))
print(len(division['test']))
print(len(division['val']))

In [ ]:
division

In [ ]:
from ochre.utils import get_chars, get_sequences

seq_length = 53

raw_val, gs_val, ocr_val = read_texts(division.get('val'), data_dir)
raw_test, gs_test, ocr_test = read_texts(division.get('test'), data_dir)
raw_train, gs_train, ocr_train = read_texts(division.get('train'), data_dir)

chars, num_chars, ci = get_chars(raw_val, raw_test, raw_train, False)

gs_seqs_val, ocr_seqs_val = get_sequences(gs_val, ocr_val, seq_length)
gs_seqs_test, ocr_seqs_test = get_sequences(gs_test, ocr_test, seq_length)
gs_seqs_train, ocr_seqs_train = get_sequences(gs_train, ocr_train, seq_length)

print('n samples val', len(gs_seqs_val))
print('n samples test', len(gs_seqs_test))
print('n samples train', len(gs_seqs_train))

In [ ]:
gs_seqs_val[0]

In [ ]:
ocr_seqs_val[0]

In [ ]:
for s in gs_seqs_test:
    print(s)

In [ ]:
def filter_ocr_space(ocr_text, gs_seqs, ocr_seqs):
    ocr_selected = []
    gs_selected = []
    indices = []
    for i, c in enumerate(ocr_text):
        if c == ' ':
            if i < len(ocr_text)-1 and ocr_text[i+1] != ' ':
                try:
                    ocr_selected.append(ocr_seqs[i+1])
                    gs_selected.append(gs_seqs[i+1])
                    #print(repr(ocr_seqs_test[i+1]))
                    indices.append(i+1)
                except IndexError:
                    break
    return gs_selected, ocr_selected, indices
gs_selected_test, ocr_selected_test, indices = filter_ocr_space(ocr_test, gs_seqs_test, ocr_seqs_test)

In [ ]:
ocr_seqs_test

In [ ]:
for gs, ocr, index in zip(gs_selected_test, ocr_selected_test, indices):
    print(gs)
    print(ocr)
    print(index)
    print()

In [ ]:
ocr_selected_test

In [ ]:
print(len(ocr_train), len(ocr_selected))

In [ ]:
len(gs_selected)

In [ ]:
gs_selected[-1]

In [ ]:
ocr_selected[-1]

In [ ]:
gs_selected_test, ocr_selected_test = filter_ocr_space(ocr_test, gs_seqs_test, ocr_seqs_test)
gs_selected_train, ocr_selected_train = filter_ocr_space(ocr_train, gs_seqs_train, ocr_seqs_train)
gs_selected_val, ocr_selected_val = filter_ocr_space(ocr_val, gs_seqs_val, ocr_seqs_val)

print('n samples val', len(gs_selected_val))
print('n samples test', len(gs_selected_test))
print('n samples train', len(gs_selected_train))

In [ ]:
import pickle

with open('train.pkl', 'wb') as f:
    pickle.dump((gs_selected_train, ocr_selected_train), f)
    
with open('val.pkl', 'wb') as f:
    pickle.dump((gs_selected_val, ocr_selected_val), f)
    
with open('ci.pkl', 'wb') as f:
    pickle.dump(ci, f)

In [ ]:
def to_space_tokenized(string):
    result = []
    for c in string:
        if c != ' ':
            result.append(c)
        else:
            result.append('<SPACE>')
    return ' '.join(result)
to_space_tokenized('Dit is een test.')

In [ ]:
def save(gs_seqs, ocr_seqs, name):
    with open('{}.ocr'.format(name), 'w') as f:
        for s in ocr_seqs:
            f.write(to_space_tokenized(s))
            f.write('\n')
    with open('{}.gs'.format(name), 'w') as f:
        for s in gs_seqs:
            f.write(to_space_tokenized(s))
            f.write('\n')

save(gs_selected_train, ocr_selected_train, 'train')
save(gs_selected_test, ocr_selected_test, 'test')
save(gs_selected_val, ocr_selected_val, 'val')

In [ ]:
# divide in words: space in ocr and gs means word boundary
import json

def get_words(aligned_file):
    with open(aligned_file, encoding='utf-8') as f:
            aligned = json.load(f)
            
    #print(len(aligned['gs']), len(aligned['ocr']))
    
    gs_words = []
    ocr_words = []

    gs_word = []
    ocr_word = []

    for gs_c, ocr_c in zip(aligned['gs'], aligned['ocr']):
        if gs_c == ' ' and ocr_c == ' ':
            gs_words.append(''.join(gs_word))
            ocr_words.append(''.join(ocr_word))
            
            gs_word = []
            ocr_word = []
        else:
            gs_word.append(gs_c)
            ocr_word.append(ocr_c)
            
    return gs_words, ocr_words
            
gs_words, ocr_words = get_words('/home/jvdzwaan/data/icdar2017st/eng_monograph/aligned/1.json')

In [ ]:
# Amrhein and Clematide use two preceding and one succeeding word as context sensitive input data (max seq. length is 53)
import itertools
import os

def ngrams(tokens, n=4):
    ngrams = zip(*[tokens[i:] for i in range(n)])
    return [" ".join(ngram).strip() for ngram in ngrams]

def create_context_data(data_files, name, max_len=53):
    
    with open('{}.gs'.format(name), 'w') as gs, open('{}.ocr'.format(name), 'w') as ocr:
        for in_file in data_files:
            gs_words, ocr_words = get_words(in_file)
            
            for ngram in ngrams(gs_words):
                gs.write(to_space_tokenized(ngram[:max_len]))
                gs.write('\n')
            
            for ngram in ngrams(ocr_words):
                ocr.write(to_space_tokenized(ngram[:max_len]))
                ocr.write('\n')
                
datasets = ('/home/jvdzwaan/data/icdar2017st/eng_monograph/datadivision.json',
            '/home/jvdzwaan/data/icdar2017st/eng_periodical/datadivision.json',
            '/home/jvdzwaan/data/icdar2017st/fr_monograph/datadivision.json',
            '/home/jvdzwaan/data/icdar2017st/fr_periodical/datadivision.json')
data_dirs = ('/home/jvdzwaan/data/icdar2017st/eng_monograph/aligned/',
             '/home/jvdzwaan/data/icdar2017st/eng_periodical/aligned/',
             '/home/jvdzwaan/data/icdar2017st/fr_monograph/aligned/',
             '/home/jvdzwaan/data/icdar2017st/fr_periodical/aligned/')


data = {'train': [], 'test': [], 'val': []}

for ds, data_dir in zip(datasets, data_dirs):
    #print(data_dir)
    with open(ds) as d:
        division = json.load(d)
    
    for n in division.keys():
        for f in division[n]:
            data[n].append(os.path.join(data_dir, f))

In [ ]:
%%time
create_context_data(data['train'], 'train-context')
create_context_data(data['test'], 'test-context')
create_context_data(data['val'], 'val-context')

In [ ]:
%%time
import os

datasets = '/home/jvdzwaan/data/icdar2017st/eng_monograph/datadivision.json'
data_dir = '/home/jvdzwaan/data/icdar2017st/eng_monograph/aligned/'

with open(datasets) as d:
    division = json.load(d)

create_context_data([os.path.join(data_dir, f) for f in division['train']], 'train-eng_mon-context')
create_context_data([os.path.join(data_dir, f) for f in division['test']], 'test-eng_mon-context')
create_context_data([os.path.join(data_dir, f) for f in division['val']], 'val-eng_mon-context')

In [ ]:
l = [1, 2, 3]
l[:2]

In [ ]:
print(v.vocabulary_)

In [ ]:
for string in ocr_selected_val:
    if len(string) > 53:
        print(string)

In [ ]:
from ochre.datagen import DataGenerator

dg_val = DataGenerator(xData=ocr_selected_val, yData=gs_selected_val, char_to_int=ci,
                       seq_length=seq_length, padding_char='\n', oov_char='@',
                       batch_size=100, shuffle=False)
dg_test = DataGenerator(xData=ocr_selected_test, yData=gs_selected_test, char_to_int=ci,
                       seq_length=seq_length, padding_char='\n', oov_char='@',
                       batch_size=100, shuffle=False)
dg_train = DataGenerator(xData=ocr_selected_train, yData=gs_selected_train, char_to_int=ci,
                       seq_length=seq_length, padding_char='\n', oov_char='@',
                       batch_size=100, shuffle=False)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import Bidirectional
from keras.layers import RepeatVector
from keras.layers import Embedding
from keras.callbacks import ModelCheckpoint

n_nodes = 1000
dropout = 0.2
n_embed = 256
n_vocab = len(ci)

loss='categorical_crossentropy'
optimizer='adam'
metrics=['accuracy']

model = Sequential()

# encoder

model.add(Embedding(n_vocab, n_embed, input_length=seq_length))
model.add(LSTM(n_nodes, input_shape=(seq_length, n_vocab)))
# For the decoder's input, we repeat the encoded input for each time step
model.add(RepeatVector(seq_length))
model.add(LSTM(n_nodes, return_sequences=True))

# For each of step of the output sequence, decide which character should be
# chosen
model.add(TimeDistributed(Dense(n_vocab, activation='softmax')))
model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

In [ ]:
# initialize saving of weights
#filepath = os.path.join(weights_dir, '{loss:.4f}-{epoch:02d}.hdf5')
filepath = '{loss:.4f}-{epoch:02d}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1,
                                 save_best_only=True, mode='min')
callbacks_list = [checkpoint]

# do training (and save weights)
model.fit_generator(dg_train, steps_per_epoch=len(dg_train), epochs=10, 
                    validation_data=dg_val, 
                    validation_steps=len(dg_val), callbacks=callbacks_list,
                    use_multiprocessing=True,
                    workers=3)